In [ ]:
!pip install opencv-python
!pip install paddleocr
!pip install paddlepaddle
!pip install easyocr
!pip install boto3
!pip install --upgrade setuptools
!pip install --no-cache-dir paddleocr
!python -m pip install paddlepaddle-gpu==2.5.0.post118 -f
!pip install paddlepaddle
!pip install paddleocr
!pip install --upgrade setuptools
!pip install transformers
!pip install --upgrade transformers
!pip install pytesseract
!pip install --upgrade jupyterlab ipython

In [ ]:
pip install setuptools==65.5.0
!pip install fire
!pip install paddleocr
rm -rf /home/ec2-user/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/*

In [ ]:
!pip install fastapi uvicorn python-multipart opencv-python-headless
!pip install pyngrok
!ngrok authtoken 2nTsHaF58Q0BnwC7r40TCJBY1oL_316Zt6vqobJVMnfqKzY7r
!pip install aiofiles

In [1]:
from PIL import Image
import cv2
import numpy as np
import json
from paddleocr import PaddleOCR
from matplotlib import pyplot as plt
import pandas as pd


In [ ]:
#image downscaling
from PIL import Image

# Increase the limit to handle large images
Image.MAX_IMAGE_PIXELS = None

def resize_to_resolution(img_path, output_image_path, resolution):
    # Open the image file
    with Image.open(img_path) as img:
        # Get original dimensions
        original_width, original_height = img.size
        print(f"Original size: {img.size}")
        
        # Get target resolution dimensions
        target_width, target_height = resolution
        
        # Calculate aspect ratio
        aspect_ratio = original_width / original_height
        
        # Adjust resolution to maintain aspect ratio
        if original_width > original_height:
            # Fit width to target and adjust height
            new_width = target_width
            new_height = int(target_width / aspect_ratio)
        else:
            # Fit height to target and adjust width
            new_height = target_height
            new_width = int(target_height * aspect_ratio)
        
        # Resize the image
        resized_img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
        
        # Save the resized image
        resized_img.save(output_image_path)
        print(f"Resized image to: {resized_img.size}")
        print(f"Image saved to {output_image_path}")
        
        # Show the resized image
        resized_img.show()

# Example usage:
# Resize the image to 2K or 4K resolution

# # For 2K (2560x1440) resolution
# resize_to_resolution(r"/home/ec2-user/Dataset/sample1.jpg", "output_image_2k.jpg", (2560, 1440))

# # For 4K (3840x2160) resolution
resize_to_resolution(r"./Dataset/sample3.jpg", "output_image_4k-23.jpg", (3840, 2160))

In [ ]:
#denoising the image

# Define the image path
path = './sample1-out/output_image_2k.jpg'

# Load the compressed image
def denoise_image(path):
    image = cv2.imread(path)

    # Check if the image was successfully loaded
    if image is None:
        print("Error: Unable to open/read image file. Check the file path and integrity.")
    else:
        # Apply Non-Local Means Denoising (correct parameters)
        # Parameters: (source, h strength, template window size, search window size)
        denoised_image_nlmeans = cv2.fastNlMeansDenoisingColored(image, h=10, templateWindowSize=7, searchWindowSize=21)

        # Apply bilateral filter for stronger noise reduction without blurring the edges
        # Parameters: (source, filter size, sigma for color space, sigma for coordinate space)
        denoised_image_bilateral = cv2.bilateralFilter(denoised_image_nlmeans, d=9, sigmaColor=100, sigmaSpace=100)

        # Apply sharpening kernel to bring back some clarity after denoising
        sharpening_kernel = np.array([[0, -1, 0],
                                    [-1, 5, -1],
                                    [0, -1, 0]])

        # Sharpen the image after denoising
        sharpened_image = cv2.filter2D(denoised_image_bilateral, -1, sharpening_kernel)

        # Convert the image to grayscale for contrast enhancement
        gray_image = cv2.cvtColor(sharpened_image, cv2.COLOR_BGR2GRAY)

        # Use histogram equalization to improve contrast
        equalized_image = cv2.equalizeHist(gray_image)

        # Convert back to BGR to save as color image
        final_image = cv2.cvtColor(equalized_image, cv2.COLOR_GRAY2BGR)
        enhanced_image_path = "enhanced_image_denoised.jpg"
        # Save the enhanced image without increasing pixel dimensions
        cv2.imwrite(enhanced_image_path, final_image)
        # print("Image processing complete. The enhanced image has been saved.")
        return enhanced_image_path


In [3]:
# Function to process image with EasyOCR and PaddleOCR
def perform_ocr_and_get_json(img_path):
    img = cv2.imread(img_path)
    easy_reader = easyocr.Reader(['en'])
    paddle_reader = PaddleOCR(use_angle_cls=True, lang='en')
    easyocr_result = easy_reader.readtext(img_path, detail=1)
    
    rows = []
    for line in easyocr_result:
        try:
            box = line[0]
            easyocr_text = line[1]
            confidence = line[2]

            # Crop and run PaddleOCR
            x_min = int(min(box[0][0], box[1][0], box[2][0], box[3][0]))
            y_min = int(min(box[0][1], box[1][1], box[2][1], box[3][1]))
            x_max = int(max(box[0][0], box[1][0], box[2][0], box[3][0]))
            y_max = int(max(box[0][1], box[1][1], box[2][1], box[3][1]))

            cropped_img = img[y_min:y_max, x_min:x_max]
            cropped_img_path = './temp_cropped_img.jpg'
            cv2.imwrite(cropped_img_path, cropped_img)

            paddle_result = paddle_reader.ocr(cropped_img_path, cls=True)

            if paddle_result[0]:
                refined_text = paddle_result[0][0][1][0]
                refined_confidence = paddle_result[0][0][1][1]
            else:
                refined_text = easyocr_text
                refined_confidence = confidence

            # Convert bounding box to native Python list of int (to avoid numpy types)
            box = [[int(point[0]), int(point[1])] for point in box]

            # Append the processed result
            rows.append({
                "bounding_box": box,
                "easyocr_text": easyocr_text,
                "paddleocr_text": refined_text,
                "confidence": refined_confidence
            })

        except TypeError as e:
            print(f"Skipping text due to TypeError: {e}")
            continue

    # Save JSON
    json_file_path = './combined_ocr_results.json'
    try:
        with open(json_file_path, 'w') as json_file:
            json.dump(rows, json_file, indent=4)
    except TypeError as e:
        print(f"Error saving JSON: {e}")

    return json_file_path

In [ ]:
# Function to process the image pipeline
# from Image_Conversion import *
def process_image_pipeline(img_path, output_image_path, resolution):
    try:
        resized_image_path = resize_to_resolution(img_path, output_image_path, resolution)
        print(f"Resized image saved at: {resized_image_path}")  # Log resized image path

        enhanced_image_path = denoise_image(resized_image_path)
        if enhanced_image_path:
            print(f"Denoised image saved at: {enhanced_image_path}")  # Log denoised image path
            json_file_path = perform_ocr_and_get_json(enhanced_image_path)
            print(f"JSON file generated at: {json_file_path}")  # Log JSON file path
            return json_file_path
        else:
            print("Error: Denoising failed.")  # Log denoising failure
            return {"error": "Image processing failed"}
    except Exception as e:
        print(f"Error in process_image_pipeline: {str(e)}")  # Log error in processing pipeline
        return {"error": str(e)}